In [ ]:
!pip install tensorflow matplotlib numpy scikit-learn seaborn nltk opencv-python

In [ ]:
# Build the Image classification model by dividing the model into following 4
# stages:
# a. Loading and preprocessing the image data
# b. Defining the model’s architecture
# c. Training the model
# d. Estimating the model’s performance

In [0]:
"""
IMAGE CLASSIFICATION MODEL - CIFAR-10 DATASET
=============================================
Implementation divided into 4 stages as required:
a. Loading and preprocessing the image data
b. Defining the model's architecture
c. Training the model
d. Estimating the model's performance
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# =============================================================================
# STAGE a: LOADING AND PREPROCESSING THE IMAGE DATA
# =============================================================================

def load_and_preprocess_data():
    """
    Stage a: Load and preprocess the CIFAR-10 dataset
    Returns preprocessed training and testing data
    """
    print("=" * 60)
    print("STAGE a: LOADING AND PREPROCESSING DATA")
    print("=" * 60)
    
    # Load CIFAR-10 dataset
    print("Loading CIFAR-10 dataset...")
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
    
    # Dataset information
    print("\nDataset Information:")
    print(f"Training images shape: {x_train.shape}")
    print(f"Training labels shape: {y_train.shape}")
    print(f"Test images shape: {x_test.shape}")
    print(f"Test labels shape: {y_test.shape}")
    
    # Class names for CIFAR-10
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                   'dog', 'frog', 'horse', 'ship', 'truck']
    
    # Display class distribution
    print("\nClass Distribution in Training Set:")
    unique, counts = np.unique(y_train, return_counts=True)
    for i, (class_id, count) in enumerate(zip(unique, counts)):
        print(f"Class {class_id} ({class_names[class_id]}): {count} samples")
    
    # Data Preprocessing Steps
    print("\nPreprocessing data...")
    
    # 1. Normalize pixel values to [0, 1]
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    
    # 2. Convert labels to categorical one-hot encoding
    num_classes = 10
    y_train_categorical = keras.utils.to_categorical(y_train, num_classes)
    y_test_categorical = keras.utils.to_categorical(y_test, num_classes)
    
    print("Preprocessing completed!")
    print(f"Normalized training data range: [{x_train.min():.3f}, {x_train.max():.3f}]")
    print(f"Training labels categorical shape: {y_train_categorical.shape}")
    
    return (x_train, y_train, y_train_categorical, 
            x_test, y_test, y_test_categorical, class_names)

# =============================================================================
# STAGE b: DEFINING THE MODEL'S ARCHITECTURE
# =============================================================================

def create_cnn_model(input_shape=(32, 32, 3), num_classes=10):
    """
    Stage b: Define a Convolutional Neural Network architecture
    Using CNN for better image classification performance
    """
    print("\n" + "=" * 60)
    print("STAGE b: DEFINING MODEL ARCHITECTURE")
    print("=" * 60)
    
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', 
                      input_shape=input_shape, name='conv1'),
        layers.BatchNormalization(name='bn1'),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv2'),
        layers.MaxPooling2D((2, 2), name='pool1'),
        layers.Dropout(0.25, name='drop1'),
        
        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv3'),
        layers.BatchNormalization(name='bn2'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv4'),
        layers.MaxPooling2D((2, 2), name='pool2'),
        layers.Dropout(0.25, name='drop2'),
        
        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv5'),
        layers.BatchNormalization(name='bn3'),
        layers.MaxPooling2D((2, 2), name='pool3'),
        layers.Dropout(0.25, name='drop3'),
        
        # Fully Connected Layers
        layers.Flatten(name='flatten'),
        layers.Dense(256, activation='relu', name='fc1'),
        layers.BatchNormalization(name='bn4'),
        layers.Dropout(0.5, name='drop4'),
        layers.Dense(128, activation='relu', name='fc2'),
        layers.Dropout(0.5, name='drop5'),
        layers.Dense(num_classes, activation='softmax', name='output')
    ])
    
    # Display model architecture
    print("Model Architecture Summary:")
    model.summary()
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("\nModel compiled successfully!")
    print("Optimizer: Adam")
    print("Loss function: Categorical Crossentropy")
    print("Metrics: Accuracy")
    
    return model

# =============================================================================
# STAGE c: TRAINING THE MODEL
# =============================================================================

def train_model(model, x_train, y_train_categorical, x_test, y_test_categorical):
    """
    Stage c: Train the model on the training data
    """
    print("\n" + "=" * 60)
    print("STAGE c: TRAINING THE MODEL")
    print("=" * 60)
    
    # Training parameters
    batch_size = 64
    epochs = 50
    validation_split = 0.2
    
    print(f"Training Parameters:")
    print(f"Batch size: {batch_size}")
    print(f"Epochs: {epochs}")
    print(f"Validation split: {validation_split}")
    
    # Callbacks for better training
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    # Train the model
    print("\nStarting training...")
    history = model.fit(
        x_train, y_train_categorical,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test_categorical),
        callbacks=callbacks,
        verbose=1,
        shuffle=True
    )
    
    print("Training completed!")
    return history

# =============================================================================
# STAGE d: ESTIMATING THE MODEL'S PERFORMANCE
# =============================================================================

def evaluate_model(model, history, x_test, y_test, y_test_categorical, class_names):
    """
    Stage d: Evaluate model performance comprehensively
    """
    print("\n" + "=" * 60)
    print("STAGE d: ESTIMATING MODEL PERFORMANCE")
    print("=" * 60)
    
    # 1. Basic Evaluation
    print("1. BASIC EVALUATION")
    test_loss, test_accuracy = model.evaluate(x_test, y_test_categorical, verbose=0)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Error Rate: {(1-test_accuracy):.4f}")
    
    # 2. Predictions
    print("\n2. MAKING PREDICTIONS")
    y_pred = model.predict(x_test, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = y_test.flatten()
    
    # 3. Detailed Classification Report
    print("\n3. DETAILED CLASSIFICATION REPORT")
    print(classification_report(y_true_classes, y_pred_classes, 
                              target_names=class_names, digits=4))
    
    # 4. Confusion Matrix
    print("4. CONFUSION MATRIX")
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix - CIFAR-10 Classification', fontsize=16, fontweight='bold')
    plt.xlabel('Predicted Labels', fontsize=12)
    plt.ylabel('True Labels', fontsize=12)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 5. Training History Visualization
    print("5. TRAINING HISTORY VISUALIZATION")
    plot_training_history(history)
    
    # 6. Sample Predictions
    print("6. SAMPLE PREDICTIONS VISUALIZATION")
    visualize_sample_predictions(model, x_test, y_true_classes, y_pred_classes, class_names)
    
    return test_accuracy, y_pred_classes

def plot_training_history(history):
    """Plot training history for accuracy and loss"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    ax1.set_title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss', linewidth=2)
    ax2.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    ax2.set_title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

def visualize_sample_predictions(model, x_test, y_true, y_pred, class_names, num_samples=12):
    """Visualize sample predictions with true vs predicted labels"""
    # Select random samples
    indices = np.random.choice(len(x_test), num_samples, replace=False)
    
    plt.figure(figsize=(15, 12))
    for i, idx in enumerate(indices):
        plt.subplot(3, 4, i + 1)
        
        # Display image
        plt.imshow(x_test[idx])
        
        # Get prediction probabilities
        pred_probs = model.predict(x_test[idx:idx+1], verbose=0)[0]
        pred_class = np.argmax(pred_probs)
        true_class = y_true[idx]
        
        # Set title color based on correctness
        color = 'green' if pred_class == true_class else 'red'
        confidence = np.max(pred_probs) * 100
        
        plt.title(f'True: {class_names[true_class]}\nPred: {class_names[pred_class]}\nConf: {confidence:.1f}%', 
                 color=color, fontweight='bold')
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('sample_predictions.png', dpi=300, bbox_inches='tight')
    plt.show()

# =============================================================================
# MAIN EXECUTION FUNCTION
# =============================================================================

def main():
    """
    Main function to execute all 4 stages of image classification
    """
    print("IMAGE CLASSIFICATION PIPELINE - CIFAR-10 DATASET")
    print("=" * 60)
    
    try:
        # Stage a: Loading and preprocessing
        (x_train, y_train, y_train_categorical, 
         x_test, y_test, y_test_categorical, class_names) = load_and_preprocess_data()
        
        # Stage b: Model architecture
        model = create_cnn_model()
        
        # Stage c: Training
        history = train_model(model, x_train, y_train_categorical, x_test, y_test_categorical)
        
        # Stage d: Performance estimation
        test_accuracy, y_pred = evaluate_model(model, history, x_test, y_test, 
                                             y_test_categorical, class_names)
        
        # Save the model
        model.save('cifar10_cnn_model.h5')
        print(f"\nModel saved as 'cifar10_cnn_model.h5'")
        
        # Final summary
        print("\n" + "=" * 60)
        print("FINAL SUMMARY")
        print("=" * 60)
        print(f"Final Test Accuracy: {test_accuracy:.4f}")
        print(f"Final Test Error Rate: {(1-test_accuracy):.4f}")
        print("Model training and evaluation completed successfully!")
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

IMAGE CLASSIFICATION PIPELINE - CIFAR-10 DATASET
STAGE a: LOADING AND PREPROCESSING DATA
Loading CIFAR-10 dataset...

Dataset Information:
Training images shape: (50000, 32, 32, 3)
Training labels shape: (50000, 1)
Test images shape: (10000, 32, 32, 3)
Test labels shape: (10000, 1)

Class Distribution in Training Set:
Class 0 (airplane): 5000 samples
Class 1 (automobile): 5000 samples
Class 2 (bird): 5000 samples
Class 3 (cat): 5000 samples
Class 4 (deer): 5000 samples
Class 5 (dog): 5000 samples
Class 6 (frog): 5000 samples
Class 7 (horse): 5000 samples
Class 8 (ship): 5000 samples
Class 9 (truck): 5000 samples

Preprocessing data...
Preprocessing completed!
Normalized training data range: [0.000, 1.000]
Training labels categorical shape: (50000, 10)

STAGE b: DEFINING MODEL ARCHITECTURE


C:\Users\Yoda ji\Favorites\timewaste\timewaste\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model Architecture Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                       │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bn1 (BatchNormalization)             │ (None, 32, 32, 32)          │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2 (Conv2D)                       │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1 (MaxPooling2D)                 │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ drop1 (Dropout)                      │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3 (Conv2D)                       │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bn2 (BatchNormalization)             │ (None, 16, 16, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv4 (Conv2D)                       │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2 (MaxPooling2D)                 │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ drop2 (Dropout)                      │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv5 (Conv2D)                       │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bn3 (BatchNormalization)             │ (None, 8, 8, 128)           │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool3 (MaxPooling2D)                 │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ drop3 (Dropout)                      │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc1 (Dense)                          │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bn4 (BatchNormalization)             │ (None, 256)                 │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ drop4 (Dropout)                      │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc2 (Dense)                          │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ drop5 (Dropout)                      │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 700,074 (2.67 MB)

 Trainable params: 699,114 (2.67 MB)

 Non-trainable params: 960 (3.75 KB)


Model compiled successfully!
Optimizer: Adam
Loss function: Categorical Crossentropy
Metrics: Accuracy

STAGE c: TRAINING THE MODEL
Training Parameters:
Batch size: 64
Epochs: 50
Validation split: 0.2

Starting training...
Epoch 1/50
138/782 ━━━━━━━━━━━━━━━━━━━━ 1:08 107ms/step - accuracy: 0.1703 - loss: 3.0658


KeyboardInterrupt


KeyboardInterrupt

